## Dynamic Time Warping
* 1NN + bespoke distance metric

In [2]:
import pandas as pd
import numpy as np
import scipy
import matplotlib
from matplotlib import pyplot as plt
import os
import re
import time
import sklearn
from glob import glob
from sklearn.neighbors import KNeighborsClassifier
os.getcwd()

'/Users/kangshuoli/Documents/VScode_workspace/GR5398/doc'

#### Calculate the optimal distance in a recursion fashion

In [ ]:
def DTW(a, b, MTSC = True, method = "adaptive"):
    '''
    Input: 
    a -> time seires, 
    b -> time series, 
    MTSC -> default is multivariate time series, 
    method: use adaptive warping (DTWa) by default, combined both independent and dependent warping

    Output:
    optimal distance
    '''
    










